In [1]:
import pandas as pd
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, normalizers
from tokenizers.processors import BertProcessing
from tokenizers.normalizers import Sequence, NFD, Lowercase, StripAccents
from tokenizers import Tokenizer
import json
import os

In [2]:
# Define the core path
PATH = 'E:/Software/Data Science and AI/NLP/Edliyye/Legal Acts Question Answering/NLP project'

text_path = os.path.join(PATH, 'full_qanun_text_chunks.parquet')

data = pd.read_parquet(text_path)

display(data.head())
display(data.tail())
data.info()
display(len(data['text'].value_counts()))

,text,unique_word_count
0,azərbaycan respublikası mülki məcəlləsinin təs...,13
1,azərbaycan respublikasının mülki məcəlləsi təs...,6
2,azərbaycan respublikasının mülki məcəlləsi qüv...,24
3,azərbaycan respublikasının mülki məcəlləsinə d...,35
4,azərbaycan ssr mülki mülki prosessual məcəlləl...,17


,text,unique_word_count
1500188,maddədə istehsal məişət tullantıları sözləri t...,8
1500189,maddədə istehsal məişət tullantıları sözləri t...,9
1500190,maddədə sənaye məişət tullantılarının sözləri ...,9
1500191,qiymətli metallar qiymətli daşlar azərbaycan r...,19
1500192,yaşıllıqların mühafizəsi azərbaycan respublika...,18


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500193 entries, 0 to 1500192
Data columns (total 2 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   text               1500193 non-null  object
 1   unique_word_count  1500193 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 22.9+ MB


967905

In [ ]:
data['text'].loc[data['text'].str.contains('azərmənzilkomplektlayihətəmir')]

In [ ]:
data['text'].iloc[15671]

In [4]:
# Function to calculate number of unique words
def count_unique_words(text_series):
    all_words = set()
    for text in text_series:
        words = text.lower().split()
        all_words.update(words)
    return len(all_words)

# Calculate number of unique words
unique_word_count = count_unique_words(data['text'])
print(f"Number of unique words: {unique_word_count}")

Number of unique words: 235331


## Training the WordPiece Tokenizer

In [3]:
# Define a tokenizer model
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

# Customize normalizer (you may adjust this based on your needs)
normalizer = Sequence([NFD(), Lowercase(), StripAccents()])
tokenizer.normalizer = normalizer

# Pre-tokenizer
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Set up the trainer with WordPiece
trainer = trainers.WordPieceTrainer(
    vocab_size=100_000,  # Adjusting vocab size
    min_frequency=10,  # Setting min frequency
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

# Write the corpus to a temporary file
corpus_file = "corpus.txt"
with open(corpus_file, 'w', encoding='utf-8') as f:
    for text in data['text']:
        f.write(text + '\n')

# Train the tokenizer
tokenizer.train([corpus_file], trainer)

# Post-processing to add CLS and SEP tokens
tokenizer.post_processor = BertProcessing(
    ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ("[CLS]", tokenizer.token_to_id("[CLS]"))
)

# Set a reasonable max length for the tokenizer before saving
tokenizer.model_max_length = 512

# Save the tokenizer
tokenizer.save("tokenizer.json")

# Cleanup the temporary corpus file
os.remove(corpus_file)

print("Tokenizer training complete. Tokenizer saved to 'tokenizer.json'. Temporary corpus file removed.")

Tokenizer training complete. Tokenizer saved to 'tokenizer.json'. Temporary corpus file removed.


## Post-processing to Remove Redundant Tokens

We can implement a filtering function to remove redundant subwords if the full word is present in the vocabulary.

In [ ]:
# Load the trained tokenizer
tokenizer = Tokenizer.from_file("tokenizer.json")

# Get the vocabulary
vocab = tokenizer.get_vocab()

# Helper function to filter out redundant subwords
def remove_redundant_subwords(vocab):
    # Create a set of full words
    full_words = {token for token in vocab.keys() if "##" not in token}
    
    # Create a new vocab dictionary without redundant subwords
    new_vocab = {token: index for token, index in vocab.items() if token in full_words or "##" not in token}
    
    return new_vocab

# Filter the vocabulary
filtered_vocab = remove_redundant_subwords(vocab)

# Save the filtered vocabulary back to the tokenizer
tokenizer.model.vocab = filtered_vocab
tokenizer.save("filtered_tokenizer.json")

print("Filtered tokenizer saved to 'filtered_tokenizer.json'.")

## Loading the Trained Tokenizer for Use

In [4]:
# Load the trained tokenizer
tokenizer = Tokenizer.from_file("tokenizer.json")

# Example usage
text = "Səninlə danışmaq istəyirəm."
encoded = tokenizer.encode(text)
print("Token IDs:", encoded.ids)
print("Tokens:", encoded.tokens)

Token IDs: [1, 348, 61, 147, 28009, 58456, 0, 2]
Tokens: ['[CLS]', 'sən', '##in', '##lə', 'danısmaq', 'istəyirəm', '[UNK]', '[SEP]']


In [5]:
data['text'].iloc[5]

'azərbaycan ssr mülki mülki prosessual məcəllələrinin qüvvəyə minməsi əlaqədar azərbaycan ssr qanunvericilik aktlarının dəyişdirilməsi qüvvədən düşmüş hesab edilməsi azərbaycan ssr ali soveti rəyasət heyətinin mart tarixli fərmanı azərbaycan ssr ali sovetinin məlumatı'

In [6]:
# Example usage
text = data['text'].iloc[5]
encoded = tokenizer.encode(text)
print("Token IDs:", encoded.ids)
print()
print("Tokens:", encoded.tokens)

Token IDs: [1, 97, 3126, 604, 604, 2221, 44158, 698, 1933, 758, 97, 3126, 464, 1101, 3051, 3751, 3220, 298, 228, 97, 3126, 989, 5052, 6866, 2207, 1401, 242, 1136, 97, 3126, 989, 2485, 2274, 2]

Tokens: ['[CLS]', 'azərbaycan', 'ssr', 'mulki', 'mulki', 'prosessual', 'məcəllələrinin', 'quvvəyə', 'minməsi', 'əlaqədar', 'azərbaycan', 'ssr', 'qanunvericilik', 'aktlarının', 'dəyisdirilməsi', 'quvvədən', 'dusmus', 'hesab', 'edilməsi', 'azərbaycan', 'ssr', 'ali', 'soveti', 'rəyasət', 'heyətinin', 'mart', 'tarixli', 'fərmanı', 'azərbaycan', 'ssr', 'ali', 'sovetinin', 'məlumatı', '[SEP]']


### Convert from the "tokenizers" library to a format compatible with the "transformers" library

In [ ]:
# Convert the custom tokenizer to a format compatible with Hugging Face
bert_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

# Save the converted tokenizer
bert_tokenizer.save_pretrained('tokenizer_directory')